In [9]:
import asf_search as asf
from shapely.geometry import Point, Polygon, shape
import pandas as pd
import geopandas as gpd
from rasterio.crs import CRS

In [3]:
wkt = Polygon([[-117.7167, 35.5909],[-117.6322, 35.5909],[-117.6322, 35.6452],[-117.7167, 35.6452],[-117.7167, 35.5909]]).wkt
opts = {
    'platform': asf.PLATFORM.SENTINEL1,
    'processingLevel': [asf.PRODUCT_TYPE.SLC],
    'beamMode': [asf.BEAMMODE.IW],
    'start': '2022-01-01T00:00:00Z',
    'end': '2022-02-01T23:59:59Z'
}
results = asf.search(intersectsWith=wkt,**opts)
print(len(results))

6


In [5]:
def format_results_for_sent1(results: list) -> gpd.GeoDataFrame:
    geometry = [shape(r.geojson()['geometry']) for r in results]
    data = [r.properties for r in results]

    df = pd.DataFrame(data)
    df = gpd.GeoDataFrame(df, geometry=geometry, crs=CRS.from_epsg(4326))

    if df.empty:
        warn('Dataframe is empty! Check inputs.')
        return df

    df['startTime'] = pd.to_datetime(df.startTime)
    df['stopTime'] = pd.to_datetime(df.stopTime)
    df['start_date'] = pd.to_datetime(df.startTime.dt.date)
    df['start_date_str'] = df.start_date.dt.date.map(str)
    df['pathNumber'] = df['pathNumber'].astype(int)
    df.drop(columns=['browse'], inplace=True)
    df = df.sort_values(by=['startTime', 'pathNumber']).reset_index(drop=True)

    return df

In [10]:
df = format_results_for_sent1(results)
df.head()

,beamModeType,bytes,centerLat,centerLon,faradayRotation,fileID,fileName,flightDirection,frameNumber,granuleType,...,processingLevel,sceneName,sensor,startTime,stopTime,temporalBaseline,url,geometry,start_date,start_date_str
0,IW,4402366634,36.2505,-117.9358,None,S1A_IW_SLC__1SDV_20220101T015100_20220101T0151...,S1A_IW_SLC__1SDV_20220101T015100_20220101T0151...,ASCENDING,114,SENTINEL_1A_FRAME,...,SLC,S1A_IW_SLC__1SDV_20220101T015100_20220101T0151...,C-SAR,2022-01-01 01:51:00,2022-01-01 01:51:27,None,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,"POLYGON ((-116.70898 37.25085, -116.38108 35.6...",2022-01-01,2022-01-01
1,IW,4379405228,35.1339,-117.5335,None,S1A_IW_SLC__1SDV_20220101T135218_20220101T1352...,S1A_IW_SLC__1SDV_20220101T135218_20220101T1352...,DESCENDING,475,SENTINEL_1A_FRAME,...,SLC,S1A_IW_SLC__1SDV_20220101T135218_20220101T1352...,C-SAR,2022-01-01 13:52:18,2022-01-01 13:52:46,None,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,"POLYGON ((-119.08044 34.48653, -118.74294 36.1...",2022-01-01,2022-01-01
2,IW,4385168449,36.2503,-117.9371,None,S1A_IW_SLC__1SDV_20220113T015100_20220113T0151...,S1A_IW_SLC__1SDV_20220113T015100_20220113T0151...,ASCENDING,114,SENTINEL_1A_FRAME,...,SLC,S1A_IW_SLC__1SDV_20220113T015100_20220113T0151...,C-SAR,2022-01-13 01:51:00,2022-01-13 01:51:27,None,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,"POLYGON ((-116.71032 37.25084, -116.38232 35.6...",2022-01-13,2022-01-13
3,IW,4361047217,35.1339,-117.5334,None,S1A_IW_SLC__1SDV_20220113T135218_20220113T1352...,S1A_IW_SLC__1SDV_20220113T135218_20220113T1352...,DESCENDING,475,SENTINEL_1A_FRAME,...,SLC,S1A_IW_SLC__1SDV_20220113T135218_20220113T1352...,C-SAR,2022-01-13 13:52:18,2022-01-13 13:52:45,None,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,"POLYGON ((-119.08031 34.48656, -118.74277 36.1...",2022-01-13,2022-01-13
4,IW,4391776002,36.25,-117.937,None,S1A_IW_SLC__1SDV_20220125T015059_20220125T0151...,S1A_IW_SLC__1SDV_20220125T015059_20220125T0151...,ASCENDING,114,SENTINEL_1A_FRAME,...,SLC,S1A_IW_SLC__1SDV_20220125T015059_20220125T0151...,C-SAR,2022-01-25 01:50:59,2022-01-25 01:51:26,None,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,"POLYGON ((-116.71029 37.25055, -116.38228 35.6...",2022-01-25,2022-01-25


In [ ]:
df.iloc[:2].to_file('test.geojson',driver='GeoJSON')